<h1>Bike - GeoFit<h1>
    
<h3> Importing Libraries <h3></h3>

In [89]:
import sys 

import pandas as pd 
import numpy as np 
import re
import IPython.display as display
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import chi2_contingency
from scipy.stats.contingency import association
import scipy.stats as st
import statsmodels.api as sm

import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

import scipy.stats as stats


from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
pd.set_option('display.max_columns', None)

<h4> Ignoring warnings <h4>

In [91]:
import warnings
warnings.filterwarnings('ignore')

<h2> Data Extraction <h2></h2>

In [93]:
df = pd.read_csv(
    "/Users/susanakohlhaas/Documents/IronHack/Final Project/geometrics.mtb-news.de.csv",
    delimiter=";",  
    encoding='utf-8'
)
df.columns = df.columns.str.strip()
print(df.columns)


print("Running inside virtual machine:", sys.prefix)

Index(['URL', 'Brand', 'Model', 'Year', 'Category', 'Motorized', 'Frame Size',
       'Frame Config', 'Wheel Size', 'Reach', 'Stack', 'STR', 'Front Center',
       'Head Tube Angle', 'Seat Tube Angle Effective', 'Seat Tube Angle Real',
       'Top Tube Length', 'Top Tube Length Horizontal', 'Head Tube Length',
       'Seat Tube Length', 'Standover Height', 'Chainstay Length', 'Wheelbase',
       'Bottom Bracket Offset', 'Bottom Bracket Height',
       'Fork Installation Height', 'Fork Offset', 'Fork Trail',
       'Suspension Travel (rear)', 'Suspension Travel (front)'],
      dtype='object')
Running inside virtual machine: /opt/anaconda3


<h5>Check general information </h5>

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6529 entries, 0 to 6528
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   URL                         6529 non-null   object 
 1   Brand                       6529 non-null   object 
 2   Model                       6529 non-null   object 
 3   Year                        6529 non-null   int64  
 4   Category                    6529 non-null   object 
 5   Motorized                   6529 non-null   bool   
 6   Frame Size                  6529 non-null   object 
 7   Frame Config                2318 non-null   object 
 8   Wheel Size                  6529 non-null   object 
 9   Reach                       6482 non-null   float64
 10  Stack                       6432 non-null   float64
 11  STR                         6432 non-null   float64
 12  Front Center                849 non-null    float64
 13  Head Tube Angle             6510 

<h5>Check Column information </h5>

In [97]:
df.head()

,URL,Brand,Model,Year,Category,Motorized,Frame Size,Frame Config,Wheel Size,Reach,Stack,STR,Front Center,Head Tube Angle,Seat Tube Angle Effective,Seat Tube Angle Real,Top Tube Length,Top Tube Length Horizontal,Head Tube Length,Seat Tube Length,Standover Height,Chainstay Length,Wheelbase,Bottom Bracket Offset,Bottom Bracket Height,Fork Installation Height,Fork Offset,Fork Trail,Suspension Travel (rear),Suspension Travel (front)
0,https://geometrics.mtb-news.de/bike/1of1-r01-2...,1of1,R.01,2023,Road,False,XXS,NaN,28″ / 700C,376.0,512.0,1.361702,NaN,71.4,NaN,74.5,NaN,520.0,112.0,460.0,731.0,408.0,972.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://geometrics.mtb-news.de/bike/1of1-r01-2...,1of1,R.01,2023,Road,False,XS,NaN,28″ / 700C,370.0,500.0,1.351351,NaN,70.8,NaN,75.0,NaN,505.0,102.0,430.0,713.0,408.0,966.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://geometrics.mtb-news.de/bike/1of1-r01-2...,1of1,R.01,2023,Road,False,S,NaN,28″ / 700C,383.0,530.0,1.383812,NaN,72.0,NaN,74.5,NaN,535.0,127.0,490.0,758.0,408.0,981.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://geometrics.mtb-news.de/bike/1of1-r01-2...,1of1,R.01,2023,Road,False,M,NaN,28″ / 700C,390.0,549.0,1.407692,NaN,72.0,NaN,73.5,NaN,550.0,147.0,520.0,785.0,408.0,985.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://geometrics.mtb-news.de/bike/1of1-r01-2...,1of1,R.01,2023,Road,False,L,NaN,28″ / 700C,397.0,571.0,1.438287,NaN,72.0,NaN,73.5,NaN,570.0,167.0,550.0,810.0,408.0,999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h5> Remove columns that are not needed </h5>

In [99]:
columns_to_remove = [
    "Motorized", "Frame Config", "Front Center", "Frame Config",
    "Seat Tube Angle Real", "Top Tube Length Horizontal", "Head Tube Length", 
    "Seat Tube Length", "Bottom Bracket Offset", "Bottom Bracket Height", 
    "Fork Installation Height", "Fork Offset", "Fork Trail", 
    "Suspension Travel (rear)", "Suspension Travel (front)"
]

df = df.drop(columns=columns_to_remove, errors='ignore')
df.head()

,URL,Brand,Model,Year,Category,Frame Size,Wheel Size,Reach,Stack,STR,Head Tube Angle,Seat Tube Angle Effective,Top Tube Length,Standover Height,Chainstay Length,Wheelbase
0,https://geometrics.mtb-news.de/bike/1of1-r01-2...,1of1,R.01,2023,Road,XXS,28″ / 700C,376.0,512.0,1.361702,71.4,NaN,NaN,731.0,408.0,972.0
1,https://geometrics.mtb-news.de/bike/1of1-r01-2...,1of1,R.01,2023,Road,XS,28″ / 700C,370.0,500.0,1.351351,70.8,NaN,NaN,713.0,408.0,966.0
2,https://geometrics.mtb-news.de/bike/1of1-r01-2...,1of1,R.01,2023,Road,S,28″ / 700C,383.0,530.0,1.383812,72.0,NaN,NaN,758.0,408.0,981.0
3,https://geometrics.mtb-news.de/bike/1of1-r01-2...,1of1,R.01,2023,Road,M,28″ / 700C,390.0,549.0,1.407692,72.0,NaN,NaN,785.0,408.0,985.0
4,https://geometrics.mtb-news.de/bike/1of1-r01-2...,1of1,R.01,2023,Road,L,28″ / 700C,397.0,571.0,1.438287,72.0,NaN,NaN,810.0,408.0,999.0


<h4> Round up Columns up</h4>

In [101]:
import numpy as np

columns_to_round_up = [
    "Wheelbase",
    "Chainstay Length",
    "Standover Height",
    "Top Tube Length",
    "Seat Tube Angle Effective",
    "Head Tube Angle",
    "STR",
    "Stack",
    "Reach"
]

for column in columns_to_round_up:
    if column in df.columns:
        df[column] = np.ceil(df[column] * 100) / 100  # Rounds up to 2 decimal places
    else:
        print(f"Warning: Column '{column}' not found in the DataFrame.")

print(df[columns_to_round_up].head())


   Wheelbase  Chainstay Length  Standover Height  Top Tube Length  \
0      972.0             408.0             731.0              NaN   
1      966.0             408.0             713.0              NaN   
2      981.0             408.0             758.0              NaN   
3      985.0             408.0             785.0              NaN   
4      999.0             408.0             810.0              NaN   

   Seat Tube Angle Effective  Head Tube Angle   STR  Stack  Reach  
0                        NaN            71.41  1.37  512.0  376.0  
1                        NaN            70.80  1.36  500.0  370.0  
2                        NaN            72.00  1.39  530.0  383.0  
3                        NaN            72.00  1.41  549.0  390.0  
4                        NaN            72.00  1.44  571.0  397.0  


<h4> Wheel Size Clean up</h4>

In [103]:
print(type(df))           # Check the type of df
print(len(df))            # If df is a list, how many elements does it have?
print(df[:5])             # Preview the first few items


<class 'pandas.core.frame.DataFrame'>
6529
                                                 URL Brand Model  Year  \
0  https://geometrics.mtb-news.de/bike/1of1-r01-2...  1of1  R.01  2023   
1  https://geometrics.mtb-news.de/bike/1of1-r01-2...  1of1  R.01  2023   
2  https://geometrics.mtb-news.de/bike/1of1-r01-2...  1of1  R.01  2023   
3  https://geometrics.mtb-news.de/bike/1of1-r01-2...  1of1  R.01  2023   
4  https://geometrics.mtb-news.de/bike/1of1-r01-2...  1of1  R.01  2023   

  Category Frame Size  Wheel Size  Reach  Stack   STR  Head Tube Angle  \
0     Road        XXS  28″ / 700C  376.0  512.0  1.37            71.41   
1     Road         XS  28″ / 700C  370.0  500.0  1.36            70.80   
2     Road          S  28″ / 700C  383.0  530.0  1.39            72.00   
3     Road          M  28″ / 700C  390.0  549.0  1.41            72.00   
4     Road          L  28″ / 700C  397.0  571.0  1.44            72.00   

   Seat Tube Angle Effective  Top Tube Length  Standover Height  \


In [105]:
df = pd.DataFrame(df)
print(type(df))
df["Wheel Size"].unique()

<class 'pandas.core.frame.DataFrame'>


array(['28″ / 700C', '27,5″ / 650B', '29″', 'Mullet 29/27,5', '26″',
       'andere', '24″', '20″', 'Mullet 27,5/26'], dtype=object)

In [ ]:
df["Wheel Size"].value_counts()

In [107]:
df["Wheel Size"].value_counts(dropna=False)
df = df.dropna(subset=["Wheel Size"])

<h4> Frame Size Clean up</h4>

In [ ]:
df["Frame Size"].unique()

In [109]:
pd.set_option('display.max_rows', None)
print(df["Frame Size"].value_counts())

Frame Size
L                1121
M                1120
XL               1088
S                 958
XS                245
XXL               184
SM                 81
MD                 78
56                 78
58                 73
54                 69
LG                 69
61                 48
51                 43
S2                 42
S3                 41
52                 39
S4                 33
S1                 33
49                 32
S5                 31
60                 30
50                 30
53                 29
Medium             28
Large              27
47                 26
ML                 25
M/L                24
55                 23
57                 21
S6                 21
XXS                20
48                 19
L/XL               18
S/M                18
62                 16
59                 16
Small              15
X-Large            14
44                 12
One Size           11
2XS                11
2XL                 8
165                 8

In [113]:
frame_sizes_to_keep = ["XXL", "XL", "L", "M", "S", "XS", "XXS"]
df_filtered = df[df["Frame Size"].isin(frame_sizes_to_keep)]
df=df_filtered

<h4> Year Clean up</h4>

In [115]:
df = df[df["Year"] != 0]
print(df["Year"].value_counts())

Year
2022    1516
2023    1214
2021    1181
2020     216
2019     154
2018     122
2024     120
2016      48
2017      47
2015      39
2014      23
2013       9
2009       4
2012       4
2010       3
2001       3
2003       3
Name: count, dtype: int64


<h5> Clean up and EDA of the rest of the columns  </h5>

In [117]:
df.isnull().sum()

URL                             0
Brand                           0
Model                           0
Year                            0
Category                        0
Frame Size                      0
Wheel Size                      0
Reach                          24
Stack                          74
STR                            74
Head Tube Angle                 8
Seat Tube Angle Effective     800
Top Tube Length              2904
Standover Height             2029
Chainstay Length               29
Wheelbase                     223
dtype: int64

In [ ]:
df.eq('').sum()

In [ ]:
print(df.columns.tolist())

In [119]:
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_values_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_values_percentage})
missing_df = missing_df[missing_df["Missing Values"] > 0]
print(missing_df)

                           Missing Values  Percentage
Top Tube Length                      2904   61.708457
Standover Height                     2029   43.115172
Seat Tube Angle Effective             800   16.999575
Wheelbase                             223    4.738632
Stack                                  74    1.572461
STR                                    74    1.572461
Chainstay Length                       29    0.616235
Reach                                  24    0.509987
Head Tube Angle                         8    0.169996


In [ ]:
df = df.round(2)
df

<h5> Clean and remove duplicates </h5>

In [ ]:
df = df.drop_duplicates()
df.duplicated().sum()

<h5> Clean up and categorise bike types </h5>

In [121]:
def categorize_bike_types(df):
    category_map = {
        "XC": "Cross Country", "TRAIL": "Trail", "ENDURO": "Enduro", "DH": "Downhill",
        "ROAD": "Gravel/Road"
    }
    if "CATEGORY" in df.columns:
        df["CATEGORY"] = df["CATEGORY"].str.upper().map(category_map).fillna("OTHER")
    return df

<h5> Standardise Brand & Model names </h5>

In [123]:
def standardize_brand_model(df):
    if "BRAND" in df.columns:
        df["BRAND"] = df["BRAND"].str.strip().str.upper()
    if "MODEL" in df.columns:
        df["MODEL"] = df["MODEL"].str.strip().str.upper()
    return df

<h5> Missing Values Heatmap</h5>


In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), cbar=False, cmap="viridis")
plt.title("Missing Values Heatmap")
plt.show()

<h5> Correlation Matrix </h5>

In [ ]:
plt.figure(figsize=(14, 8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()


<h5> Feature Distributions </h5>

In [ ]:
df.hist(figsize=(14, 10), bins=30, edgecolor="black")
plt.suptitle("Feature Distributions", fontsize=16)
plt.show()

<h5> Boxplots of Numerical Features </h5>

In [ ]:
plt.figure(figsize=(14, 8))
df.boxplot(rot=90)  # Rotate labels for readability
plt.title("Boxplots of Numerical Features")
plt.show()

<h5> Reach vs Stack by Bike Category (Sampled 1000) </h5>

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(x="Reach", y="Stack", hue="Category", data=df.sample(1000, random_state=42), alpha=0.7)
plt.title("Reach vs Stack by Bike Category (Sampled 1000)")
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
<h5> Distribution of Wheel Sizes </h5>

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(y=df["Wheel Size"], order=df["Wheel Size"].value_counts().index, palette="coolwarm")
plt.title("Distribution of Wheel Sizes")
plt.xlabel("Count")
plt.ylabel("Wheel Size")
plt.show()

In [ ]:
<h5> Wheel Size Distribution by Bike Type </h5>

In [ ]:
plt.figure(figsize=(10, 5))
wheel_type_counts = df.groupby(["Wheel Size", "Category"]).size().unstack()
wheel_type_counts.plot(kind="barh", figsize=(10, 5), colormap="coolwarm", edgecolor="black")

plt.title("Wheel Size Distribution by Bike Type")
plt.ylabel("Wheel Size")
plt.xlabel("Count")
plt.legend(title="Bike Type")
plt.grid(axis="x", linestyle="--", alpha=0.7)
plt.show()

In [ ]:
<h5> Distribution of Standard Frame Sizes </h5>

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=df, x="Frame Size", order=df["Frame Size"].value_counts().index)
plt.xticks(rotation=45)
plt.title("Distribution of Frame Sizes")
plt.show()


<h5> Reach vs Stack (Bike Fit) </h5>

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(data=df, x="Reach", y="Stack", hue="Frame Size")
plt.title("Reach vs Stack (Bike Fit)")
plt.show()


<h5> Head Tube Angle Over Time </h5>

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(data=df, x="Year", y="Head Tube Angle")
plt.title("Head Tube Angle Over Time")
plt.show()


<h5> Head Tube Angle Over Time by Category </h5>

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(data=df, x="Year", y="Head Tube Angle", hue="Category")
plt.title("Head Tube Angle Over Time by Category")
plt.show()

<h4> Statisticaly Analysis </h4>

<h4>Wheel Size Trends Over Years - ANOVO</h4>

In [138]:

def clean_wheel_size(value):
    cleaned_value = re.sub(r'[^\d\.]', '', str(value))
    try:
        return float(cleaned_value)
    except ValueError:
        return np.nan  

# ✅ Group wheel sizes by year for ANOVA
wheel_sizes_by_year = [group["Wheel Size"].values for _, group in df.groupby("Year")]

# ✅ Perform one-way ANOVA
f_stat, p_value = stats.f_oneway(*wheel_sizes_by_year)

# ✅ Display results
print("Wheel Size Trends Over Years (ANOVA)")
print(f"F-Statistic: {f_stat:.2f}")
print(f"P-Value: {p_value:.5f}")
if p_value < 0.05:
    print("Significant difference in wheel size across years.")


Wheel Size Trends Over Years (ANOVA)
F-Statistic: 35.84
P-Value: 0.00000
Significant difference in wheel size across years.


In [140]:
## The ANOVA test shows a statistically significant difference in wheel size across years, with an F-statistic of 35.84 and a p-value less than 0.00001. This means the mean wheel size has changed significantly over time. The high F-statistic indicates substantial differences between years relative to within-year variability.

<h4>Frame Size Trends Over Years - ANOVO</h4>

In [155]:
df = df.dropna(subset=["Frame Size", "Year"])

df["Frame Size"] = pd.to_numeric(df["Frame Size"], errors='coerce')
df = df.dropna(subset=["Frame Size"])

frame_sizes_by_year = [group["Frame Size"].values for _, group in df.groupby("Year")]

f_stat, p_value = stats.f_oneway(*frame_sizes_by_year)

print("Frame Size Trends Over Years (ANOVA)")
print(f"F-Statistic: {f_stat:.2f}")
print(f"P-Value: {p_value:.5f}")
if p_value < 0.05:
    print("Significant difference in frame size across years.")


Frame Size Trends Over Years (ANOVA)
F-Statistic: 5.34
P-Value: 0.00000
Significant difference in frame size across years.


In [157]:
## The ANOVA test indicates a statistically significant difference in frame size across years, with an F-statistic of 5.34 and a p-value less than 0.00001. This suggests that the mean frame size has changed significantly over time.

<h4>T- Test for Wheelbase (Mountain vs. Road Bikes)- ANOVO</h4>

In [174]:
t_stat, p_value = stats.ttest_ind(mountain_bikes, road_bikes, equal_var=False)

print("T-Test: Wheelbase Comparison (Mountain vs. Road)")
print(f"T-Statistic: {t_stat:.2f}")
print(f"P-Value: {p_value:.5f}")
if p_value < 0.05:
        print("Significant difference in wheelbase between Mountain and Road bikes.")


T-Test: Wheelbase Comparison (Mountain vs. Road)
T-Statistic: 19.40
P-Value: 0.00000
Significant difference in wheelbase between Mountain and Road bikes.


In [ ]:
## The T-test shows a statistically significant difference in wheelbase between mountain and road bikes, with a t-statistic of 19.40 and a p-value less than 0.00001. This indicates that the mean wheelbase of mountain bikes is significantly different from that of road bikes.

<h4>Save motified DF to a new CSV file to use for Tableau and Streamlit</h4>

In [ ]:
new_filename = f"/Users/susanakohlhaas/Documents/IronHack/Final Project/geometrics_modifiedv3.csv"

# Save the modified DataFrame to a new CSV file
df.to_csv(new_filename, sep=';', index=False)
